<a href="https://colab.research.google.com/github/rohrl/llm_shenanigans/blob/main/soft_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# based on https://github.com/kipgparker/soft-prompt-tuning/blob/main/example.ipynb

In [2]:
!pip install sentencepiece transformers accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoModelForCausalLM, AutoTokenizer

In [4]:
import torch
import torch.nn as nn

In [5]:
torch.set_default_device('cuda')

In [6]:


# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# model = GPT2LMHeadModel.from_pretrained('gpt2', device_map="cuda")

# model_id = "itsliupeng/llama2_7b_mmlu"
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", device_map="cuda", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model.device

device(type='cuda', index=0)

## Sanity check

In [8]:
def create_empty_input():
    return {'input_ids': torch.empty(size=(1,0)).to(torch.int64), 'attention_mask': torch.empty(size=(1,0)).to(torch.int64)}

In [9]:
sanity_text = "The capital of Australia is"
sanity_output = model.generate(input_ids = tokenizer.encode(sanity_text, return_tensors="pt"), max_length=15, num_return_sequences=1)
print("*******************************************\n" +
      tokenizer.decode(sanity_output[0], skip_special_tokens=True) +
      "\n*******************************************\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


*******************************************
The capital of Australia is Canberra.
    How many more tickets should Is
*******************************************



In [10]:
# inputs = create_empty_input()
# model.generate(inputs['input_ids'], max_length=10, num_return_sequences=1)
# print("==================\n" + tokenizer.decode(sanity_output[0], skip_special_tokens=True) + "\n==================\n")

## Soft Embeddings

In [11]:
class SoftEmbedding(nn.Module):
    def __init__(self,
                wte: nn.Embedding,
                n_tokens: int = 10,
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to

        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens,
                                                                               random_range,
                                                                               initialize_from_vocab))

    def initialize_embedding(self,
                             wte: nn.Embedding,
                             n_tokens: int = 10,
                             random_range: float = 0.5,
                             initialize_from_vocab: bool = True):
        """initializes learned embedding

        Args:
            same as __init__

        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            # this takes first n_tokens words from vocab and uses as init of learnt embeddings
            return self.wte.weight[:n_tokens].clone().detach()
        # .half() is needed for Phi2
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range).half().to('cuda')

    def forward(self, tokens):
        """run forward pass

        Args:
            tokens (torch.long): input tokens before encoding

        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        # below line means that first n_tokens tokens will be ignored (?)
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [12]:
# How many soft prompt tokens do we want to use.
num_soft_prompt_tokens = 20
initialize_from_vocab = False  # True

In [13]:
orig_embeddings = model.get_input_embeddings()
orig_embeddings

Embedding(51200, 2560)

In [14]:
tokenizer

CodeGenTokenizerFast(name_or_path='microsoft/phi-2', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50260: AddedToken("                            ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50261: AddedToken("            

In [15]:
s_wte = SoftEmbedding(model.get_input_embeddings(),
                      n_tokens = num_soft_prompt_tokens,
                      initialize_from_vocab = initialize_from_vocab)

In [16]:
s_wte

SoftEmbedding(
  (wte): Embedding(51200, 2560)
)

In [17]:
model.set_input_embeddings(s_wte)

In [18]:
def prepend_with_soft_prompts_padding(inputs, num_soft_tokens, pad_token_id = tokenizer.unk_token_id, labels = None):
    """
    Need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens,
    even though it does not matter what you pad input_ids with, it's just to make HF happy.
    More exp: the SoftEmbedding implementation ignores first num_soft_prompt_tokens of input tokens so this padding is to insert them at the beginning (and also make consistent with attention_mask length)
    Padding is made of repeated "unk_token" (but it doesn't matter as it's ignored).
    """
    batch_size = inputs['input_ids'].size(0)

    inputs['input_ids'] = torch.cat([torch.full((batch_size, num_soft_tokens), pad_token_id), inputs['input_ids']], 1)
    inputs['attention_mask'] = torch.cat([torch.full((batch_size, num_soft_tokens), 1), inputs['attention_mask']], 1)

    if labels is None:
        return inputs
    else:
        labels = torch.cat([torch.full((batch_size, num_soft_tokens), pad_token_id), labels], 1)
        return inputs, labels


## Inference

In [19]:
model.eval()

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): SoftEmbedding(
      (wte): Embedding(51200, 2560)
    )
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (fi

In [20]:
inputs = tokenizer("The capital of Australia is", return_tensors="pt")
# inputs = create_empty_input()

In [21]:
inputs

{'input_ids': tensor([[ 464, 3139,  286, 4505,  318]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [22]:
tokenizer.decode(inputs['input_ids'].squeeze(), skip_special_tokens=False)

'The capital of Australia is'

In [23]:
inputs = prepend_with_soft_prompts_padding(inputs, num_soft_prompt_tokens)

print(inputs)
print(tokenizer.decode(inputs['input_ids'].squeeze(), skip_special_tokens=False))

{'input_ids': tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286,  4505,   318]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>The capital of Australia is


In [24]:
# outputs = model(**inputs)

def generate_output(curr_inputs, new_out_tokens = 10):

  new_token_id = 0
  outputs = torch.cat([inputs['input_ids'], torch.full((1, new_out_tokens), 0) ], 1)

  model.eval()

  with torch.no_grad():
    for i in range(new_out_tokens):

      # outputs = model.generate(**inputs, max_length = curr_inputs['input_ids'].size(1) + 1)
      raw_outputs = model(**curr_inputs)
      # print(raw_outputs.logits.shape)

      # new_token_id = outputs.squeeze()[-1]
      new_token_id = raw_outputs.logits[:,-1,:].argmax(axis=-1).item()
      outputs[:, (-new_out_tokens+i)] = new_token_id
      # print(outputs)

      # add the new token to inputs and repeat
      curr_inputs['input_ids'] = torch.cat([curr_inputs['input_ids'], torch.full((1, 1), new_token_id)], 1)
      curr_inputs['attention_mask'] = torch.cat([curr_inputs['attention_mask'], torch.full((1,1), 1)], 1)

  return outputs



In [25]:
outputs = generate_output(inputs, new_out_tokens = 10)

In [26]:
# print(outputs.logits.shape)
print(outputs)

predicted_token_ids = outputs.squeeze()

tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286,  4505,   318, 33452,    13,   198,   198, 46344,
           657,    25,   383,  3376,  3038]], device='cuda:0')


In [27]:
text = tokenizer.decode(predicted_token_ids, skip_special_tokens=True) #[0]

# Print the decoded text
print(f"|{text}|")

|The capital of Australia is Canberra.

Solution 0: The correct option|


## Training

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy

# this token is ignored in loss - used to mask remainder of output
ignored_token_id = tokenizer.unk_token_id

target = "The capital of Australia is Honolulu."

target_tokens = tokenizer(target, return_tensors="pt")
target_len = target_tokens['input_ids'].size(1)

print(target_tokens['input_ids'])

# create the batch by repeating tokens, then in the loop mask endings

target_tokens['input_ids'] = target_tokens['input_ids'].repeat(target_len - 1, 1)
target_tokens['attention_mask'] = target_tokens['attention_mask'].repeat(target_len - 1, 1)

# labels will be the next token, so clone and left-shift 1 hop
labels = target_tokens['input_ids'].clone()
labels = labels.roll(-1, dims=-1)

# add masks
for i in range(target_len - 1):
  # pad right of i
  #labels.append(target_tokens['input_ids'][i, i+1].item())
  target_tokens['input_ids'][i, i+1:] = torch.full((1, target_len - i - 1), ignored_token_id)
  labels[i, i+1:] = torch.full((1, target_len - i - 1), ignored_token_id)
  target_tokens['attention_mask'][i, i+1:] = torch.full((1, target_len - i - 1), 0)

# last token will never be fed as input so trim all tensors
target_tokens['input_ids'] = target_tokens['input_ids'][:, :-1]
target_tokens['attention_mask'] = target_tokens['attention_mask'][:, :-1]
labels = labels[:, :-1]

# Finally, pad inputs with soft prompts
target_tokens, labels = prepend_with_soft_prompts_padding(target_tokens, num_soft_prompt_tokens, labels = labels)


print(target_tokens['input_ids'])
print(target_tokens['attention_mask'])
print(labels)


tensor([[  464,  3139,   286,  4505,   318, 43296,    13]], device='cuda:0')
tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464, 50256, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139, 50256, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286, 50256, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
           464,  3139,   286,  4505, 50256, 50256],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
    

In [29]:
# freeze entire model, then unfreeze soft embeddings
model.requires_grad_(False)
s_wte.requires_grad_(True)

SoftEmbedding(
  (wte): Embedding(51200, 2560)
)

In [58]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

class LossWithEmbReg(nn.Module):
  """
  TODO explain
  """
  def __init__(self, orig_emb: nn.Embedding, soft_emb: SoftEmbedding, target_loss: nn.Module, lamb):
    super(LossWithEmbReg, self).__init__()
    self.orig_emb = orig_emb
    self.soft_emb = soft_emb
    self.target_loss = target_loss
    # lambda reg param
    self.lamb = lamb
    self.init_nearest_n()

  def init_nearest_n(self):
    emb_tensors = self.orig_emb(torch.arange(0, self.orig_emb.num_embeddings))
    emb_tensors = emb_tensors.detach().cpu().numpy()
    self.nearest_n = NearestNeighbors(metric='cosine')
    self.nearest_n.fit(emb_tensors)

  def calc_emb_dist(self):
    X = self.soft_emb.learned_embedding.data.clone().detach().cpu().numpy()
    distances_np, _ = self.nearest_n.kneighbors(X, n_neighbors=1, return_distance=True)
    distances_np = distances_np.squeeze(axis=1)
    print(distances_np.shape)
    return np.sum(distances_np ** 2)

  def forward(self, input, target):

    loss = self.target_loss(input, target)
    reg = self.lamb * self.calc_emb_dist()

    self.last_reg = reg

    # print(f"Loss = {loss}, reg = {reg}")

    return loss + reg

In [59]:
def compute_loss(criterion, logits, labels):

    logits_flat = logits.view(-1, logits.size(-1))
    # print(logits_flat.shape)

    labels_flat = labels.flatten()
    # print(labels_flat.shape)

    loss = criterion(logits_flat, labels_flat)

    loss_per_batch = loss.mean()

    return loss_per_batch

In [60]:
# Training loop.
# Note we don't need/want an eval set - unlike in typical training, we do want to overfit to train data as much as possible.

model.train()

# criterion = nn.CrossEntropyLoss(ignore_index = ignored_token_id, reduction='none')

# add Regularization to keep soft embeddings as close to actual as possible
criterion = LossWithEmbReg(orig_embeddings,
                           s_wte,
                           nn.CrossEntropyLoss(ignore_index = ignored_token_id, reduction='none'),
                           0.01)

optimizer = optim.SGD(model.parameters(), lr=0.01) # TODO: Try Adam

best_loss = 1e9
best_soft_prompts = None

# Train the model
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(input_ids = target_tokens['input_ids'], attention_mask = target_tokens['attention_mask'])

    # print(outputs.logits.shape) #, outputs.logits)
    # print(labels.shape, labels)

    loss = compute_loss(criterion, outputs.logits, labels)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Save best
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_soft_prompts = copy.deepcopy(s_wte) # .clone() fails :(
        print('--- NEW BEST: Epoch: {}, Loss: {:.4f}'.format(epoch+1, best_loss))


    # Print the loss
    if epoch % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


--- NEW BEST: Epoch: 1, Loss: 0.6390
Epoch [1/1000], Loss: 0.6390
--- NEW BEST: Epoch: 2, Loss: 0.6219
--- NEW BEST: Epoch: 6, Loss: 0.5908
--- NEW BEST: Epoch: 10, Loss: 0.5859
Epoch [11/1000], Loss: 0.6205
Epoch [21/1000], Loss: 0.6092
--- NEW BEST: Epoch: 27, Loss: 0.5740
Epoch [31/1000], Loss: 0.6077
--- NEW BEST: Epoch: 36, Loss: 0.5688
Epoch [41/1000], Loss: 0.6041
Epoch [51/1000], Loss: 0.5943
Epoch [61/1000], Loss: 0.6030
--- NEW BEST: Epoch: 68, Loss: 0.5656
Epoch [71/1000], Loss: 0.6054
Epoch [81/1000], Loss: 0.5766
--- NEW BEST: Epoch: 83, Loss: 0.5646
Epoch [91/1000], Loss: 0.5817
Epoch [101/1000], Loss: 0.5975
Epoch [111/1000], Loss: 0.5690
--- NEW BEST: Epoch: 121, Loss: 0.5630
Epoch [121/1000], Loss: 0.5630
Epoch [131/1000], Loss: 0.5735
--- NEW BEST: Epoch: 133, Loss: 0.5589
--- NEW BEST: Epoch: 134, Loss: 0.5412
Epoch [141/1000], Loss: 0.5838
Epoch [151/1000], Loss: 0.5684
--- NEW BEST: Epoch: 156, Loss: 0.5384
Epoch [161/1000], Loss: 0.5861
Epoch [171/1000], Loss: 0.5

In [ ]:
best_loss

In [ ]:
best_loss

In [ ]:
best_soft_prompts

In [ ]:
# Set the best soft prompts on the model.
model.set_input_embeddings(best_soft_prompts)

## Test
Now see if the soft prompts changed the output.

In [ ]:
model.eval()
inputs = tokenizer("The capital of Australia is", return_tensors="pt")
inputs = prepend_with_soft_prompts_padding(inputs, num_soft_prompt_tokens)

outputs = generate_output(inputs, new_out_tokens = 10)
print("*******************************************\n" +
      tokenizer.decode(outputs.squeeze(), skip_special_tokens=True) +
      "\n*******************************************\n")


## Analysis


In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

emb_tensors = orig_embeddings(torch.arange(0, orig_embeddings.num_embeddings))
emb_tensors = emb_tensors.detach().cpu().numpy()

print(emb_tensors)

nn = NearestNeighbors(metric='cosine')

nn.fit(emb_tensors)

X = best_soft_prompts.learned_embedding.data.clone().detach().cpu().numpy()
print(X)

n_neighbors=5
all_nn_tokens = nn.kneighbors(X, n_neighbors=n_neighbors, return_distance=False)

all_nn_tokens = torch.from_numpy(all_nn_tokens).transpose(0, 1) # .squeeze(1)
print(all_nn_tokens)

# tokenizer.decode(all_nn_tokens)
[tokenizer.decode(nth_nearest_tokens) for nth_nearest_tokens in all_nn_tokens]

# TODO: plot t-sne.

Now swap embeddings back to the original ones (so we have the original model) and see if the tokens that are nearest to the learned embeddings can do the trick.

In [ ]:
model.set_input_embeddings(orig_embeddings)

sanity_text = "The capital of Australia is"
input_ids = tokenizer.encode(sanity_text, return_tensors="pt")
print(input_ids)
input_ids = torch.cat([all_nn_tokens[0], input_ids])
print(input_ids)


sanity_output = model.generate(input_ids = input_ids, max_length=35, num_return_sequences=1)
print("*******************************************\n" +
      tokenizer.decode(sanity_output[0], skip_special_tokens=True) +
      "\n*******************************************\n")


In [ ]:
# USING HF LIBRARY
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./model_checkpoints",  # Output directory for checkpoints
#     num_train_epochs=3,  # Total number of training epochs
#     per_device_train_batch_size=16,  # Batch size per device
#     per_device_eval_batch_size=16,  # Batch size for evaluation
#     warmup_steps=500,  # Number of warmup steps
#     logging_steps=100,  # Number of steps between logging
#     save_steps=1000,  # Number of steps between saving checkpoints
#     evaluation_strategy="steps",  # Evaluation strategy
#     eval_steps=1000,  # Number of steps between evaluations
# )

# trainer = Trainer(
#     model=model,  # The model to train
#     args=training_args,  # Training arguments
#     train_dataset=train_dataset,  # Training dataset
#     eval_dataset=eval_dataset,  # Evaluation dataset
# )

# trainer.train()